In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)
import keras

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = tf.keras.datasets.mnist

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
train_data, test_data = train_data / 255.0, test_data / 255.0

# Add a channels dimension
train_data = train_data[..., tf.newaxis].astype("float32")
test_data = test_data[..., tf.newaxis].astype("float32")


VALIDATION_SIZE = 5000
validation_data = train_data[:VALIDATION_SIZE, :, :, :]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, :, :, :]
train_labels = train_labels[VALIDATION_SIZE:]
params = [32, 32, 64, 64, 200, 200]
batch_size = 128

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((validation_data, validation_labels))
val_dataset = val_dataset.batch(batch_size)

class MNIST:
    def __init__(self, train_data, train_labels, validation_data, validation_labels, test_data, test_labels):
        self.train_data = train_data
        self.train_labels = train_labels
        self.test_data = test_data
        self.test_labels = test_labels
        self.validation_data = validation_data
        self.validation_labels = validation_labels

mnist_data = MNIST(train_data, train_labels, validation_data, validation_labels, test_data, test_labels)


def show(img):
    """
    Show MNSIT digits in the console.
    """
    remap = "  .*#"+"#"*100
    img = (img.flatten()+.5)*3
    if len(img) != 784: return
    print("START")
    for i in range(28):
        print("".join([remap[int(round(x))] for x in img[i*28:i*28+28]]))

def get_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(params[0], (3, 3),
                            input_shape=data.train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(params[1], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(params[2], (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(params[3], (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(params[4]))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(params[5]))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

def get_madry_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(32, (5, 5), padding="same",
                            input_shape=data.train_data.shape[1:]))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(Conv2D(64, (5, 5), padding="same"))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

    model.add(Flatten())
    model.add(Dense(1024))
    # model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

def get_madry_little_diff_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(32, (3, 3),
                            input_shape=data.train_data.shape[1:]))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024))
    # model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

def get_madry_with_relu_model(data, file_name, params, num_epochs=50, batch_size=128, train_temp=1, init=None):
    """
    Standard neural network training procedure.
    """
    model = Sequential()

    print(data.train_data.shape)
    
    model.add(Conv2D(32, (5, 5), padding="same",
                            input_shape=data.train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(Conv2D(64, (5, 5), padding="same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
basic_model = get_model(mnist_data, "mnistModel", params)

import keras
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9)
# Instantiate a loss function.
loss_fn1 = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn2 = keras.losses.BinaryCrossentropy()
loss_fn3 = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


# @tf.function
def train_step(x1, y):
    with tf.GradientTape() as tape:
        logits1 = basic_model(x1, training=True)
        loss_value = loss_fn1(y, logits1)
    grads = tape.gradient(loss_value, basic_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, basic_model.trainable_weights))
    train_acc_metric.update_state(y, logits1)
    return loss_value

# @tf.function
def test_step(x, y):
    val_logits = basic_model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

(55000, 28, 28, 1)


In [ ]:
print(basic_model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
activation_14 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
activation_15 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3136)             

In [4]:
# Training model
import time
import math 
std = 0.01
epochs = 5

total_steps = sum(1 for _ in train_dataset)
for epoch in range(epochs):
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        # Compute loss
        loss_value = train_step(x_batch_train, y_batch_train)
        print('\r', "Epoch %d" % (epoch,), 'Training step:', step+1, f'/{total_steps}', 'Loss:', float(loss_value), 'Acc:', float(train_acc_metric.result()), end='')
    
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("\nTraining acc over epoch: %.4f" % (float(train_acc),), end=' ')

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),), end=' ')
    print("Time taken: %.2fs\n" % (time.time() - start_time))

 Epoch 0 Training step: 430 /430 Loss: 0.22438444197177887 Acc: 0.7936000227928162
Training acc over epoch: 0.7936 Validation acc: 0.9682 Time taken: 42.85s

 Epoch 1 Training step: 430 /430 Loss: 0.047443702816963196 Acc: 0.9617090821266174
Training acc over epoch: 0.9617 Validation acc: 0.9824 Time taken: 12.76s

 Epoch 2 Training step: 430 /430 Loss: 0.01357968058437109 Acc: 0.9728545546531677
Training acc over epoch: 0.9729 Validation acc: 0.9822 Time taken: 12.57s

 Epoch 3 Training step: 430 /430 Loss: 0.21093526482582092 Acc: 0.9776545166969299
Training acc over epoch: 0.9777 Validation acc: 0.9858 Time taken: 12.39s

 Epoch 4 Training step: 430 /430 Loss: 0.026699639856815338 Acc: 0.9820181727409363
Training acc over epoch: 0.9820 Validation acc: 0.9866 Time taken: 12.31s



In [5]:
from google.colab import drive
drive.mount('/content/drive')
tf.keras.models.save_model(basic_model, '/content/drive/MyDrive/CS726/basic_model')

Mounted at /content/drive


In [ ]:
x_tensor1 = tf.convert_to_tensor(train_data[1:3,:,:,:].reshape(-1,784))
y1 = train_labels[1:3]
with tf.GradientTape() as t:
    t.watch(x_tensor1)
    x_input = tf.reshape(x_tensor1, [-1,28,28,1])
    logits11 = basic_model(x_input, training=True)
    loss_value3 = loss_fn3(y1, logits11)
    loss_value1 = loss_fn1(y1, logits11)
result = loss_value1
grad = t.gradient(loss_value3, x_tensor1)
print(loss_value1, loss_value3)

tf.Tensor(0.02583846, shape=(), dtype=float32) tf.Tensor(0.05167692, shape=(), dtype=float32)


In [ ]:
print(grad)

tf.Tensor(
[[-2.3741059e-06 -2.4822975e-06 -4.2244160e-06 ...  6.7994904e-08
  -7.2581656e-07 -1.2402419e-06]
 [ 8.1715118e-03  6.3168872e-03  6.6806478e-03 ...  3.4910217e-03
   1.2533757e-03 -4.7204274e-04]], shape=(2, 784), dtype=float32)


In [ ]:
print(grad.shape)

(2, 784)


In [6]:
"""
Implementation of attack methods. Running this file as a program will
apply the attack to the model specified by the config file and store
the examples in an .npy file.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# import tensorflow as tf
# import numpy as np


class LinfPGDAttack:
  def __init__(self, model, epsilon, k, a, random_start, loss_func):
    """Attack parameter initialization. The attack performs k steps of
       size a, while always staying within epsilon from the initial
       point."""
    self.model = model
    self.epsilon = epsilon
    self.k = k
    self.a = a
    self.rand = random_start

  def perturb(self, x_nat, y):
    """Given a set of examples (x_nat, y), returns a set of adversarial
       examples within epsilon of x_nat in l_infinity norm."""
    if self.rand:
        x = x_nat + np.random.uniform(-self.epsilon, self.epsilon, x_nat.shape)
        x = np.clip(x, 0, 1) # ensure valid pixel range
    else:
        x = np.copy(x_nat)

    for i in range(self.k):
        x_tensor = tf.convert_to_tensor(x)
        with tf.GradientTape() as t:
            t.watch(x_tensor)
            x_image = tf.reshape(x_tensor, [-1,28,28,1])
            logits1 = basic_model(x_image, training=True)
            loss_value = loss_fn3(y, logits1)
        result = loss_value
        grad = t.gradient(loss_value, x_tensor)
        
        x += self.a * np.sign(grad)

        x = np.clip(x, x_nat - self.epsilon, x_nat + self.epsilon) 
        x = np.clip(x, 0, 1) # ensure valid pixel range

    return x

In [ ]:
import json
import sys
import math


attack = LinfPGDAttack(basic_model,
                        0.3,
                        40,
                        0.01,
                        True,
                        "xent")



# Iterate over the samples batch-by-batch
num_eval_examples = 10000
eval_batch_size = 128
num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

x_adv = [] # adv accumulator

print('Iterating over {} batches'.format(num_batches))

for ibatch in range(num_batches):
    bstart = ibatch * eval_batch_size
    bend = min(bstart + eval_batch_size, num_eval_examples)
    print('batch size: {}'.format(bend - bstart))

    x_batch = test_data[bstart:bend, :, :, :].reshape(-1,784)
    y_batch = test_labels[bstart:bend]

    x_batch_adv = attack.perturb(x_batch, y_batch).reshape(-1,28,28,1)
    x_adv.append(x_batch_adv)

print('Storing examples')
path = 'pgd_attack2.npy'
x_adv = np.concatenate(x_adv, axis=0)
np.save(path, x_adv)
print('Examples stored in {}'.format(path))


In [8]:
pgd_adv = np.load('pgd_attack2.npy')

In [ ]:
i = 0
x_linf = pgd_adv[i,:,:,:]
show(x_linf)

import matplotlib.pyplot as plt

im_linf = x_linf.reshape(28, 28)
plt.gray()
plt.imshow(im_linf)
plt.figure()
plt.imshow(test_data[i,:,:,:].reshape(28,28))
print(test_labels[i])
print(np.sum((x_linf-test_data[i,:,:,:])**2))

In [10]:
def test_attack_single_input(adv, x, true_label, num_trials=10, std=0.01):
    # original input
    y = basic_model(x)
    pred_label = np.argmax(y)
    count = np.zeros(10)
    for i in range(num_trials):
        x_noisy = x + np.random.normal(scale=std, size=x.shape)
        y_noisy = basic_model(x_noisy)
        noisy_label = np.argmax(y_noisy)
        count[noisy_label] += 1
    noisy_label = np.argmax(count)

    # adv. input
    y_adv = basic_model(adv)
    adv_label = np.argmax(y_adv)
    count = np.zeros(10)
    for i in range(num_trials):
        adv_noisy = adv + np.random.normal(scale=std, size=adv.shape)
        y_noisy_adv = basic_model(adv_noisy)
        noisy_label_adv = np.argmax(y_noisy_adv)
        count[noisy_label_adv] += 1
    adv_noisy_label = np.argmax(count)
    print('\r', 'true:', true_label, 'pred:', pred_label, 'noisy_pred:', noisy_label, 'adv_pred:', adv_label, 'adv_noisy_pred:', adv_noisy_label, end='')

    return true_label, pred_label, noisy_label, adv_label, adv_noisy_label

def test_attack_multiple_inputs(num_runs=100):
    corr = 0
    noisy_corr = 0
    adv_corr = 0
    adv_noisy_corr = 0
    for run in range(num_runs):
        i = np.random.randint(0, 10000)
        x = test_data[i,:,:,:][np.newaxis,:,:,:]
        adv = pgd_adv[i,:,:,:][np.newaxis,:,:,:]
        true, pred, noisy_pred, adv_pred, adv_noisy_pred = test_attack_single_input(adv, x, test_labels[i], num_trials=10, std=0.1)
        corr += (pred == true)
        noisy_corr += (true == noisy_pred)
        adv_corr += (true == adv_pred)
        adv_noisy_corr += (true == adv_noisy_pred)
    print(f'\nAcc Original: {corr/num_runs}, Acc Adv: {adv_corr/num_runs}, Acc Adv Noisy: {adv_noisy_corr/num_runs}, Acc Ori Noisy: {noisy_corr/num_runs}')


In [ ]:
a = 1
a += (1==2)

In [ ]:
print(a)

In [11]:
test_attack_multiple_inputs()

 true: 4 pred: 4 noisy_pred: 4 adv_pred: 9 adv_noisy_pred: 9
Acc Original: 1.0, Acc Adv: 0.02, Acc Adv Noisy: 0.02, Acc Ori Noisy: 1.0
